In [45]:
from plotly import express as ex

In [46]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow import math
import numpy as np

In [47]:
def normalise_prices(prices, m: float):
    prs = np.array(prices, dtype=np.float32) / m
    return prs

In [267]:
def profit(P: tf.Variable, lbd1: tf.Variable, lbd2: tf.Variable, C):
    entropy = -tf.tensordot(P, math.log(P), axes=1)
    cnd1 = (math.reduce_sum(P) - 1)**2
    cnd2 = (tf.tensordot(P, C, axes=1) - 1)**2
    return entropy - lbd1 * 100 * cnd1 - lbd2 * 100 * cnd2

In [268]:
n = 6
prices = [1, 3, 9, 10, 100, 1000]
m = 10.0

In [269]:
C = tf.constant(normalise_prices(prices, m))
P = tf.Variable(np.ones(n, dtype=np.float32) / n)
LB1 = tf.Variable(1.0)
LB2 = tf.Variable(1.0)

In [271]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
# opt = tf.keras.optimizers.experimental.Ftrl()

In [272]:
P / tf.reduce_sum(P)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667], dtype=float32)>

In [276]:
# k = 0.0001
for i in range(100):
    with tf.GradientTape(persistent=True) as g:
        val = -profit(P, LB1, LB2, C)
    # P.assign_add(g.gradient(val, P) * k)
    # LB1.assign_add(g.gradient(val, LB1) * k)
    # LB2.assign_add(g.gradient(val, LB2) * k)
    opt.minimize(val, [P, LB1, LB2], g)
    P.assign(P / tf.reduce_sum(P))

In [277]:
print(P.numpy())
print(profit(P, LB1, LB2, C).numpy())
print(math.reduce_sum(P).numpy())
print(tf.tensordot(P, C, axes=1).numpy())

[0.16665421 0.16666542 0.16666916 0.16666934 0.16667086 0.16667102]
283221.25
1.0
18.717148


In [278]:
ex.scatter(y=P.numpy(), x=C.numpy(), log_x=True)

In [42]:
g.gradient(val, LB1)


<tf.Tensor: shape=(), dtype=float32, numpy=-0.5876293>

In [43]:
g.gradient(val, LB2)

<tf.Tensor: shape=(), dtype=float32, numpy=-29.000166>

In [41]:
g.gradient(val,P)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([  -1.5793862,   -1.7988977,   -2.4592953,   -2.569539 ,
        -12.525416 , -112.17468  ], dtype=float32)>